# Financial Markets Prices
---
### Autor: Diego Alegría   2023-02-24 v3
#### Basado en código de J Gonzalez
---
Código para obtener valores de ~ 257 precios relevantes para diversos reportes.
Utilizado todos los viernes después de que cierre la bolsa. (4 pm en Nueva York, 2 o 3 en Guate)

Usos primarios son un reporte semanal de Promerica el viernes, y para reporte de Commodities el lunes.

- Referencias para arreglo:
https://github.com/JECSand/yahoofinancials/blob/master/yahoofinancials/__init__.py
https://github.com/JECSand/yahoofinancials/issues/105
https://github.com/JECSand/yahoofinancials/commit/62119be23fe81f89fd5fb1bd450fc2088667c8df

---
### Instrucciones de actualizacion.

Solamente actualizar yahoofinancials con pip, El autor JECSand implementó los arreglos propuestos por GitHub.

- 257 precios son actualizados por medio de yahoofinancials.
- 10 variables son actualizadas manualmente, utilizando el excel Markets_Temporal.xlsx ubicado en DB\Weekly\Viernes Temporales\

Hay una seccion en este codigo al final que realiza el procesamiento para subir estas ultimas variables sin tener que abrir SQL.

# Importación de liberías

In [14]:
# import investpy
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from yahoofinancials import YahooFinancials
from datetime import*
import urllib
import numpy as np
import time
import pyodbc
import traceback
import requests as requests
import numpy as np
import sys
from sqlalchemy import*
from IPython.display import display

from pandas import Timestamp
style.use('ggplot')

#Fecha de actual: (Lugar para Hardcoded abajo)
fecha_inicial=datetime.today().strftime('%Y-%m-%d')
fecha_final=(datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

# Funciones par SQL

Información importante sobre esta celda

- El código de lectura trae de regreso 3 querys que se utilizan en este notebook
  - El primer query trae de regreso la última fecha 
  - El segundo trae una serie de fechas que son las necesarias obtener en la base de datos para homologar si se agrega un índice nuevo
  - El tercero es un sample para crear el frame que se subirá a la base de datos

In [15]:
driver= '{ODBC Driver 17 for SQL Server}'
DATABASE_CONFIG = {
    'server': 'datacabisvr2.database.windows.net',
    'database': 'CABI_DB',
    'username': 'eelvira',
    'password': 'Cabi2023!'
}

server= 'datacabisvr2.database.windows.net'
database='CABI_DB'
username= 'dalegria'
password= 'Cabi2019'

connection_string = 'DRIVER=' + driver + \
                     ';SERVER=' + server + \
                    ';PORT=1435' + \
                    ';DATABASE=' + database + \
                    ';UID=' + username + \
                    ';PWD=' + password

def write2db(df,table, fast = True):
    # prepare SQLAlchemy engine
    # Doc: https://docs.sqlalchemy.org/en/latest/dialects/mssql.html#module-sqlalchemy.dialects.mssql.pyodbc
    # same connection string as before
    params = urllib.parse.quote_plus(connection_string)
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params, fast_executemany=fast)
    flag=1
    # df columns names need to match the table column names in DB

    # check #rows in table before write operation
    sql_conn = pyodbc.connect(connection_string)
    cursor = sql_conn.cursor()
    query = '''
                SELECT count(*)
                FROM [dbo].['''+table+"]"

    try:
        cursor.execute(query)
        rows_before = cursor.fetchone()
        flag=1
    except:
        print('No table, creating a new table')
        flag=0


    # write results to Azure DB table
    df.to_sql(table, con = engine, if_exists = 'append', index = False)
    if(flag==1):
        # check #rows in table after write operation
        cursor.execute(query)
        rows_after = cursor.fetchone()
        print(rows_after)
        print("Inserted {} rows into table {}".format(rows_after[0]-rows_before[0], table))

    else:
        print('New table created')
    return


server= 'datacabisvr2.database.windows.net'
database='CABI_DB'
username= 'eelvira'
password= 'Cabi2023!'
driver= '{ODBC Driver 17 for SQL Server}'
connection_string = 'DRIVER=' + driver + \
                     ';SERVER=' + server + \
                    ';PORT=1435' + \
                    ';DATABASE=' + database + \
                    ';UID=' + username + \
                    ';PWD=' + password

def read_data_to_df(query):
    try:
        print("    \u2713 connecting db: " + database)
        sql_conn = pyodbc.connect(connection_string)
        # execute query and save data in pandas df
        df = pd.read_sql(query, sql_conn)
        return df
    except Exception as error:
        print("    \u2717 accesing {} has error".format(database))
        print("    \u2717 error message: {}".format(error))
        # I found that traceback prints much more detailed error message
        traceback.print_exc()

if __name__ == "__main__":

    # Query para tomar el último dato del dji
    query_fecha = '''
            SELECT        MAX(DatoFecha)
            FROM            dbo.TablaDatos
            WHERE        (IdIndiceEspecifico = 2988)


                '''

    query_fechas = '''
            SELECT        DatoFecha
            FROM            dbo.TablaDatos
            WHERE        (IdIndiceEspecifico = 2986)


                '''

    querydf = '''
SELECT
      TablaDatos.[IdIndiceEspecifico]
      ,[DatoFecha]
      /*,[D_Open]
      ,[D_High]
      ,[D_Low]*/
      ,[D_Close]

      ,Indice_Especifico.Nombre
      /*,Indice_Especifico.Descripcion
      ,Indice_Especifico.IdIndicador*/

  FROM [dbo].[TablaDatos]
  INNER JOIN Indice_Especifico
  ON TablaDatos.IdIndiceEspecifico=Indice_Especifico.IdIndiceEspecifico
  WHERE Indice_Especifico.IdIndicador=21
  OR Indice_Especifico.IdIndicador=22
  OR Indice_Especifico.IdIndicador=30
  OR Indice_Especifico.IdIndicador=37
            '''

## Estas tres consultas sirven para definir varios parámetros de los datos a subir.
# esto incluye fechas,
last_date = read_data_to_df(query_fecha)


fechas = read_data_to_df(query_fechas)

commdf=read_data_to_df(querydf)

last_date_value=last_date.loc[0][0]
last_date_value=datetime.strftime(last_date_value,'%Y-%m-%d')
last_date_value=pd.to_datetime(last_date_value)
last_date_value

    ✓ connecting db: CABI_DB


C:\Users\lijv1\AppData\Local\Temp\ipykernel_21448\1374698685.py:77: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, sql_conn)


    ✓ connecting db: CABI_DB
    ✓ connecting db: CABI_DB


C:\Users\lijv1\AppData\Local\Temp\ipykernel_21448\1374698685.py:135: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_date_value=last_date.loc[0][0]


Timestamp('2025-11-21 00:00:00')

---
# Lectura de Tickets ()

Si se agrega un nuevo índice se debe agregar a esta base de datos

In [16]:
df = pd.read_excel('CARGA NUEVOS ETFs SQL 2020.xlsx', sheet_name='TickersDownload')
df

,IdTD,IdTicker,Source,Descripcion,IdIndiceEspecifico
0,$BCOM(D),AW=F,Yahoo,Blmbg Commodity Index,3006
1,$BCOMEN(D),$BCOMEN(D),Pendiente,Blmbg Energy Sub,3775
2,$BCOMGR(D),$BCOMGR(D),Investing,Blmbg Grains Subindex,3007
3,$BCOMIN(D),$BCOMIN(D),Investing,Blmbg Industrial Mtls Sub,3008
4,$BCOMPR(D),$BCOMPR(D),Investing,Blmbg Precious Metals Sub,3009
...,...,...,...,...,...
261,XRT(HB),XRT,Yahoo,SPDR S&P Retail,2964
262,ZO=F,ZO=F,Yahoo,Oats Continuous Contract [Sep21],3022
263,ZR=F,ZR=F,Yahoo,Rough Rice,3998
264,DC=F,DC=F,Yahoo,Class III Milk,4020


## Unificación final y empacado de datos para base de datos 

Aquí se unifican (Cuando está disponible) los índices de Yahoo e Investing y se empacan en el formato de TablaDatos

In [17]:
df_y = pd.read_excel('FinancialMarketsBU.xlsx')
df_y

,IdIndiceEspecifico,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest
0,3006,2025-11-28,108.080002,110.680000,109.160004,110.410004,374,110.410004
1,2990,2025-11-28,23291.589844,23365.789062,23250.509766,23365.689453,4541070000,23365.689453
2,2986,2025-11-28,6822.520020,6850.859863,6819.750000,6849.089844,2558540000,6849.089844
3,3011,2025-11-28,423.000000,448.500000,443.500000,447.750000,78853,447.750000
4,3012,2025-11-28,5068.000000,5585.000000,5191.000000,5404.000000,14801,5404.000000
...,...,...,...,...,...,...,...,...
242,2962,2025-11-28,235.539993,236.800003,235.539993,236.460007,2413600,236.460007
243,2963,2025-11-28,96.059998,97.250000,95.480003,97.089996,1767000,97.089996
244,2964,2025-11-28,84.690002,84.690002,84.029999,84.349998,2152500,84.349998
245,3022,2025-11-28,288.250000,318.500000,313.000000,317.500000,137,317.500000


In [18]:
df_final=df_y.copy()
df_final.to_excel('FinancialMarkets.xlsx',index=False)
# display(df_final)

#Ordenar
df_final=df_final[['IdIndiceEspecifico',
     'DatoFecha',
     'D_Open',
     'D_High',
     'D_Low',
     'D_Close',
     'D_Volume',
     'D_OpenInterest']]
#Renombrar
df_final.columns=['IdIndiceEspecifico',
     'DatoFecha',
     'D_Open',
     'D_High',
     'D_Low',
     'D_Close',
     'D_Volume',
     'D_OpenInterest']
df_final.reset_index(inplace=True,drop=True)
df_f=df_final.copy() #Copia a final
df_f.reset_index(inplace=True,drop=True)


df_f.DatoFecha=df_final.loc[1,'DatoFecha']


In [19]:
df_f

,IdIndiceEspecifico,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest
0,3006,2025-11-28,108.080002,110.680000,109.160004,110.410004,374,110.410004
1,2990,2025-11-28,23291.589844,23365.789062,23250.509766,23365.689453,4541070000,23365.689453
2,2986,2025-11-28,6822.520020,6850.859863,6819.750000,6849.089844,2558540000,6849.089844
3,3011,2025-11-28,423.000000,448.500000,443.500000,447.750000,78853,447.750000
4,3012,2025-11-28,5068.000000,5585.000000,5191.000000,5404.000000,14801,5404.000000
...,...,...,...,...,...,...,...,...
242,2962,2025-11-28,235.539993,236.800003,235.539993,236.460007,2413600,236.460007
243,2963,2025-11-28,96.059998,97.250000,95.480003,97.089996,1767000,97.089996
244,2964,2025-11-28,84.690002,84.690002,84.029999,84.349998,2152500,84.349998
245,3022,2025-11-28,288.250000,318.500000,313.000000,317.500000,137,317.500000


## Escritura a la base de datos

Se escribe a TablaDatos y TablaDatos_BU hasta que se complete la migración

In [20]:
# df_f['DatoFecha'] = pd.to_datetime('2025-11-15')
df_f

,IdIndiceEspecifico,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest
0,3006,2025-11-28,108.080002,110.680000,109.160004,110.410004,374,110.410004
1,2990,2025-11-28,23291.589844,23365.789062,23250.509766,23365.689453,4541070000,23365.689453
2,2986,2025-11-28,6822.520020,6850.859863,6819.750000,6849.089844,2558540000,6849.089844
3,3011,2025-11-28,423.000000,448.500000,443.500000,447.750000,78853,447.750000
4,3012,2025-11-28,5068.000000,5585.000000,5191.000000,5404.000000,14801,5404.000000
...,...,...,...,...,...,...,...,...
242,2962,2025-11-28,235.539993,236.800003,235.539993,236.460007,2413600,236.460007
243,2963,2025-11-28,96.059998,97.250000,95.480003,97.089996,1767000,97.089996
244,2964,2025-11-28,84.690002,84.690002,84.029999,84.349998,2152500,84.349998
245,3022,2025-11-28,288.250000,318.500000,313.000000,317.500000,137,317.500000


In [21]:
df_f['DatoFecha'] = pd.to_datetime(df_f['DatoFecha'], errors='coerce')
df_f['DatoFecha'][0]

Timestamp('2025-11-28 00:00:00')

In [22]:
last_date_value

Timestamp('2025-11-21 00:00:00')

In [23]:
if df_f['DatoFecha'][0] > last_date_value:
    print("Nuevo dato, ingresando...")
    df_f['DatoFecha'] = pd.to_datetime(df_f['DatoFecha'], errors='coerce').dt.strftime('%Y-%m-%d')
    print(df_f.dtypes)
    display(df_f)
    write2db(df_f, "TablaDatos")
else:
    print("No hay nuevo dato")

Nuevo dato, ingresando...
IdIndiceEspecifico      int64
DatoFecha              object
D_Open                float64
D_High                float64
D_Low                 float64
D_Close               float64
D_Volume                int64
D_OpenInterest        float64
dtype: object


,IdIndiceEspecifico,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest
0,3006,2025-11-28,108.080002,110.680000,109.160004,110.410004,374,110.410004
1,2990,2025-11-28,23291.589844,23365.789062,23250.509766,23365.689453,4541070000,23365.689453
2,2986,2025-11-28,6822.520020,6850.859863,6819.750000,6849.089844,2558540000,6849.089844
3,3011,2025-11-28,423.000000,448.500000,443.500000,447.750000,78853,447.750000
4,3012,2025-11-28,5068.000000,5585.000000,5191.000000,5404.000000,14801,5404.000000
...,...,...,...,...,...,...,...,...
242,2962,2025-11-28,235.539993,236.800003,235.539993,236.460007,2413600,236.460007
243,2963,2025-11-28,96.059998,97.250000,95.480003,97.089996,1767000,97.089996
244,2964,2025-11-28,84.690002,84.690002,84.029999,84.349998,2152500,84.349998
245,3022,2025-11-28,288.250000,318.500000,313.000000,317.500000,137,317.500000


(390993,)
Inserted 247 rows into table TablaDatos
